<a href="https://colab.research.google.com/github/LoveDatax/RetailProject/blob/main/RetailProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lower, upper, trim, to_date, date_format, sum, regexp_replace, count, avg
from pyspark.sql.types import DecimalType
spark=SparkSession.builder\
.appName('Retail data project')\
.getOrCreate()

In [3]:
retails=spark.read.csv('/content/drive/MyDrive/Retail_Transactions_Dataset.csv',
                       header=True,
                       inferSchema=True,
                       sep=',',
                       escape='"',
                       quote='"',
                       multiLine=True,
                       mode='PERMISSIVE')
retails.printSchema()

root
 |-- Transaction_ID: integer (nullable = true)
 |-- Date: timestamp (nullable = true)
 |-- Customer_Name: string (nullable = true)
 |-- Product: string (nullable = true)
 |-- Total_Items: integer (nullable = true)
 |-- Total_Cost: double (nullable = true)
 |-- Payment_Method: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Store_Type: string (nullable = true)
 |-- Discount_Applied: boolean (nullable = true)
 |-- Customer_Category: string (nullable = true)
 |-- Season: string (nullable = true)
 |-- Promotion: string (nullable = true)



In [4]:
retails.show(10, truncate=False)

+--------------+-------------------+-----------------+-------------------------------------------------------------+-----------+----------+--------------+-------------+-----------------+----------------+-----------------+------+--------------------------+
|Transaction_ID|Date               |Customer_Name    |Product                                                      |Total_Items|Total_Cost|Payment_Method|City         |Store_Type       |Discount_Applied|Customer_Category|Season|Promotion                 |
+--------------+-------------------+-----------------+-------------------------------------------------------------+-----------+----------+--------------+-------------+-----------------+----------------+-----------------+------+--------------------------+
|1000000000    |2022-01-21 06:27:29|Stacey Price     |['Ketchup', 'Shaving Cream', 'Light Bulbs']                  |3          |71.65     |Mobile Payment|Los Angeles  |Warehouse Club   |true            |Homemaker        |Winter|None

In [5]:
retails_processed=retails\
.select(col("Transaction_ID").alias("transaction_id"),
    col("Date").alias("transaction_ts"),
    col("Customer_Name").alias("customer_name"),
    col("Product").alias("product"),
    col("Total_Items").alias("total_items"),
    col("Total_Cost").alias("total_cost"),
    col("Payment_Method").alias("payment_method"),
    col("City").alias("city"),
    col("Store_Type").alias("store_type"),
    col("Discount_Applied").alias("discount_applied"),
    col("Customer_Category").alias("customer_category"),
    col("Season").alias("season"),
    col("Promotion").alias("promotion"))\
.filter(col('transaction_id').isNotNull())\
.filter(col('transaction_ts').isNotNull())\
.filter(col('total_items') > 0)\
.filter(col('total_cost') >= 0)\
.withColumn('total_cost', regexp_replace(col('total_cost'), '[^0-9.]', '').cast(DecimalType(12,2)))\
.withColumn('customer_name', upper(trim(col('customer_name'))))\
.withColumn('payment_method', upper(trim(col('payment_method'))))\
.withColumn('city', upper(trim(col('city'))))\
.withColumn('store_type', upper(trim(col('store_type'))))\
.withColumn('customer_category', upper(trim(col('customer_category'))))\
.withColumn('season', upper(trim(col('season'))))\
.withColumn('promotion', upper(trim(col('promotion'))))\
.withColumn('transaction_time', date_format(col('transaction_ts'),'HH:mm:ss'))\
.withColumn('transaction_date', to_date(col('transaction_ts')))\
.drop('transaction_ts')\
.dropDuplicates()

In [6]:
retails_processed.show(20, truncate=False)

+--------------+-------------------+----------------------------------------------------------------------------+-----------+----------+--------------+-------------+-----------------+----------------+-----------------+------+--------------------------+----------------+----------------+
|transaction_id|customer_name      |product                                                                     |total_items|total_cost|payment_method|city         |store_type       |discount_applied|customer_category|season|promotion                 |transaction_time|transaction_date|
+--------------+-------------------+----------------------------------------------------------------------------+-----------+----------+--------------+-------------+-----------------+----------------+-----------------+------+--------------------------+----------------+----------------+
|1000000690    |VICTORIA BROWN     |['Soap', 'Toilet Paper', 'Peanut Butter', 'Vinegar', 'Trash Cans']          |2          |56.31     |MOB

In [7]:
city_revenue = retails_processed\
    .groupBy('city')\
    .agg(sum('total_cost').alias('total_revenue'))\
    .orderBy(col('total_revenue').desc())

city_revenue.show()

+-------------+-------------+
|         city|total_revenue|
+-------------+-------------+
|       DALLAS|   5277111.53|
|       BOSTON|   5263307.96|
|      CHICAGO|   5263187.45|
|     NEW YORK|   5252469.92|
|      HOUSTON|   5247054.78|
|SAN FRANCISCO|   5241099.86|
|        MIAMI|   5240498.44|
|      SEATTLE|   5235365.43|
|  LOS ANGELES|   5232393.19|
|      ATLANTA|   5202731.84|
+-------------+-------------+



In [8]:
store_type_revenue = retails_processed\
    .groupBy('store_type')\
    .agg(sum('total_cost').alias('total_revenue')) \
    .orderBy(col('total_revenue').desc())

store_type_revenue.show()

+-----------------+-------------+
|       store_type|total_revenue|
+-----------------+-------------+
|         PHARMACY|   8766679.01|
|      SUPERMARKET|   8763455.21|
|   WAREHOUSE CLUB|   8760029.03|
|CONVENIENCE STORE|   8731901.36|
| DEPARTMENT STORE|   8731555.57|
|  SPECIALTY STORE|   8701600.22|
+-----------------+-------------+



In [9]:
promotion_performance = retails_processed\
    .filter(col('promotion').isNotNull())\
    .groupBy('promotion')\
    .agg(sum('total_cost').alias('promotion_revenue'))\
    .orderBy(col('promotion_revenue').desc())

promotion_performance.show()

+--------------------+-----------------+
|           promotion|promotion_revenue|
+--------------------+-----------------+
|                NONE|      17554038.81|
|DISCOUNT ON SELEC...|      17462227.94|
|BOGO (BUY ONE GET...|      17438953.65|
+--------------------+-----------------+



In [15]:
customer_value = retails_processed \
    .groupBy('customer_category') \
    .agg(
        count('*').alias('transaction_count'),
        avg('total_cost').alias('avg_spend'),
        sum('total_cost').alias('total_spend')
    ) \
    .orderBy(col('total_spend').desc())
customer_value.show()

+-----------------+-----------------+---------+-----------+
|customer_category|transaction_count|avg_spend|total_spend|
+-----------------+-----------------+---------+-----------+
|         TEENAGER|           125319|52.529091| 6582893.18|
|        HOMEMAKER|           125418|52.461417| 6579605.97|
|   SENIOR CITIZEN|           125485|52.342672| 6568220.19|
|          RETIREE|           125072|52.435589| 6558223.95|
|          STUDENT|           124842|52.487994| 6552706.16|
|     PROFESSIONAL|           124651|52.525762| 6547388.77|
|      YOUNG ADULT|           124577|52.448246| 6533845.14|
|      MIDDLE-AGED|           124636|52.411318| 6532337.04|
+-----------------+-----------------+---------+-----------+



In [16]:
payment_usage = retails_processed\
    .groupBy('payment_method')\
    .count()\
    .orderBy(col('count').desc())
payment_usage.show()

+--------------+------+
|payment_method| count|
+--------------+------+
|          CASH|250230|
|    DEBIT CARD|250074|
|   CREDIT CARD|249985|
|MOBILE PAYMENT|249711|
+--------------+------+



In [17]:
season_performance = retails_processed\
    .groupBy('season')\
    .agg(sum('total_cost').alias('season_revenue'))\
    .orderBy(col('season_revenue').desc())
season_performance.show()

+------+--------------+
|season|season_revenue|
+------+--------------+
|  FALL|   13136913.71|
|SUMMER|   13116675.79|
|SPRING|   13113238.75|
|WINTER|   13088392.15|
+------+--------------+



In [20]:
discount_impact = retails_processed\
    .groupBy('discount_applied')\
    .agg(count('*').alias('transaction_count'),
        sum('total_cost').alias('total_revenue'),
        avg('total_cost').alias('avg_transaction_value'))
discount_impact.show()

+----------------+-----------------+-------------+---------------------+
|discount_applied|transaction_count|total_revenue|avg_transaction_value|
+----------------+-----------------+-------------+---------------------+
|            true|           500104|  26248916.26|            52.486915|
|           false|           499896|  26206304.14|            52.423512|
+----------------+-----------------+-------------+---------------------+



In [21]:
top_product = retails_processed\
.groupBy('product')\
.agg(sum('total_cost').alias('revenue'))\
.orderBy(col('revenue').desc())\
.limit(10)

top_product.show()

+--------------+---------+
|       product|  revenue|
+--------------+---------+
|['Toothpaste']|253689.01|
|     ['Honey']|135409.85|
| ['Deodorant']|133082.77|
|     ['Pasta']|131765.69|
|      ['Eggs']|131565.71|
|   ['Vinegar']|131538.39|
|      ['Soap']|131183.28|
|    ['Razors']|130956.57|
|    ['Salmon']|130940.12|
|       ['Jam']|130617.22|
+--------------+---------+



In [23]:
customer_buying_metrics = retails_processed\
.groupBy('customer_category')\
  .agg(avg('total_items').alias('avg_items_per_transaction'))

customer_buying_metrics.show()

+-----------------+-------------------------+
|customer_category|avg_items_per_transaction|
+-----------------+-------------------------+
|         TEENAGER|        5.489670361238121|
|          RETIREE|         5.50546085454778|
|      YOUNG ADULT|       5.4955489376048545|
|      MIDDLE-AGED|        5.501388041978241|
|   SENIOR CITIZEN|        5.493525122524605|
|          STUDENT|        5.496787939956104|
|        HOMEMAKER|       5.5024637611826055|
|     PROFESSIONAL|       5.4826595855628915|
+-----------------+-------------------------+



In [24]:
from pyspark.sql.functions import hour

peak_buying_hour = retails_processed\
.withColumn('hour', hour('transaction_time'))\
  .groupBy('hour')\
  .agg(count('*').alias('transactions'))\
  .orderBy('hour')

peak_buying_hour.show()

+----+------------+
|hour|transactions|
+----+------------+
|   0|       41416|
|   1|       41224|
|   2|       41617|
|   3|       41917|
|   4|       41768|
|   5|       41574|
|   6|       41508|
|   7|       41521|
|   8|       41926|
|   9|       41841|
|  10|       42021|
|  11|       41154|
|  12|       41790|
|  13|       41817|
|  14|       41739|
|  15|       41819|
|  16|       41505|
|  17|       41655|
|  18|       41843|
|  19|       41646|
+----+------------+
only showing top 20 rows


In [22]:
from pyspark.sql.functions import when

discount_percentage_revenue = retails_processed\
.agg(
    (sum(when(col('discount_applied') == True, col('total_cost')))/
     sum(col('total_cost')) * 100).alias('discounted_revenue_pct'))

discount_percentage_revenue.show()

+----------------------+
|discounted_revenue_pct|
+----------------------+
|       50.040617616011|
+----------------------+

